# Imports

In [ ]:
import numpy as np # linear get_custom_objects().update({'swish': Activation(swish)})algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datatable as dt

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer

from numpy.random import seed
seed(112021)
import tensorflow as tf
from tensorflow.keras import layers
tf.random.set_seed(112021)
from tensorflow import keras
from keras import backend as K
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
from keras.backend import sigmoid

def swish(x, beta = 1):
    return (x * sigmoid(beta * x))
get_custom_objects().update({'swish': Activation(swish)})

# Load dataset

In [ ]:
%%time
train = dt.fread('../input/tabular-playground-series-nov-2021/train.csv').to_pandas()
test  = dt.fread('../input/tabular-playground-series-nov-2021/test.csv').to_pandas()
sub   = dt.fread('../input/tabular-playground-series-nov-2021/sample_submission.csv').to_pandas()

In [ ]:
train.drop('id',axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [ ]:
y = train['target']
train.drop('target',axis=1,inplace=True)

# Model architecture

In [ ]:
def get_model():
    input_1 = layers.Input(shape=(train.values.shape[-1]), name="continuous")
    x = layers.Embedding(input_dim=128, output_dim=4)(input_1)
    x = layers.TimeDistributed(layers.Dense(64, activation='swish'))(x)
    x = layers.TimeDistributed(layers.Dense(64, activation='swish'))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='swish')(x)
    x = layers.Dense(256, activation='swish')(x)
    output = layers.Dense(1, activation="sigmoid", name="output")(x)
    
    model = tf.keras.Model(input_1, output)
    return model

model = get_model()
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["AUC"])
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
cb_es = tf.keras.callbacks.EarlyStopping(monitor="val_auc", patience=3, mode="max", restore_best_weights=True, verbose=1)
cb_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc", factor=0.5, patience=2, min_lr=0.000001, verbose=1)

# 5-fold model training and prediction

In [ ]:
n_folds = 5
kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=112020)
nn_oof = np.zeros(train.shape[0])

predictions = np.zeros(len(test))

for fold, (trn_idx, val_idx) in enumerate(kf.split(train)):
    print(f'Training fold {fold + 1}')
    X_train, X_test = train.iloc[trn_idx], train.iloc[val_idx]
    y_train, y_test = y.iloc[trn_idx], y.iloc[val_idx]
    
    scaler = MinMaxScaler(feature_range=(0, 1)) 
    x_nn_train = scaler.fit_transform(X_train.values)
    
    x_nn_test = scaler.transform(X_test.values)
    
    model.fit([x_nn_train], 
              y_train,               
              batch_size=2048,
              epochs=1000,
              validation_data=([x_nn_test], y_test),
              callbacks=[cb_es, cb_lr],
              validation_batch_size=len(y_test),
              shuffle=True,
              verbose = 2)
    
    preds = model.predict(x_nn_test).reshape(1,-1)[0]
    nn_oof[val_idx] = preds
    roc = roc_auc_score(y_test, preds)
    print(f" roc_auc_score: {roc}")
    print("-"*50)
    
    test_nn = scaler.transform(test.values)
    
    predictions += model.predict(test_nn).reshape(1,-1)[0].clip(0,1e10) / kf.n_splits

# Store submission and oof

In [ ]:
sub['target'] = predictions
sub.to_csv(f'submission_nn_1.csv',index = False)
np.savez_compressed('oof_nn.npz', nn_oof)